In [1]:
import pandas as pd
df = pd.read_csv('titanic_train.csv')
df.head()

c:\Users\ahmad\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\ahmad\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\ahmad\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df.sample()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
651,1,2,female,18.0,0,1,23.0,S


In [3]:
df.isna().mean()

Survived    0.000000
Pclass      0.000000
Sex         0.000000
Age         0.198653
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002245
dtype: float64

In [4]:
from sklearn.model_selection import train_test_split
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,male,NaN,1,1,15.2458,C
439,2,male,31.0,0,0,10.5000,S
840,3,male,20.0,0,0,7.9250,S
720,2,female,6.0,0,1,33.0000,S
39,3,female,14.0,1,0,11.2417,C
...,...,...,...,...,...,...,...
433,3,male,17.0,0,0,7.1250,S
773,3,male,NaN,0,0,7.2250,C
25,3,female,38.0,1,5,31.3875,S
84,2,female,17.0,0,0,10.5000,S


In [6]:
from sklearn.impute import SimpleImputer
imputer1 = SimpleImputer(strategy='median')
imputer2 = SimpleImputer(strategy='most_frequent')

X_train[['Age']] = imputer1.fit_transform(X_train[['Age']])
X_test[['Age']] = imputer1.transform(X_test[['Age']])

X_train[['Embarked']] = imputer2.fit_transform(X_train[['Embarked']])
X_test[['Embarked']] = imputer2.transform(X_test[['Embarked']])

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

scaler = StandardScaler()
encoder = OneHotEncoder(drop='first', sparse_output=False)

ct = ColumnTransformer([
    ('scaler', scaler, ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
    ('encoder', encoder, ['Sex', 'Embarked'])
]).set_output(transform='pandas')


X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)

In [8]:
X_train_scaled.head()

,scaler__Pclass,scaler__Age,scaler__SibSp,scaler__Parch,scaler__Fare,encoder__Sex_male,encoder__Embarked_Q,encoder__Embarked_S
331,-1.614136,1.253641,-0.470722,-0.479342,-0.078684,1.0,0.0,1.0
733,-0.400551,-0.477284,-0.470722,-0.479342,-0.377145,1.0,0.0,1.0
382,0.813034,0.215086,-0.470722,-0.479342,-0.474867,1.0,0.0,1.0
704,0.813034,-0.246494,0.379923,-0.479342,-0.476230,1.0,0.0,1.0
813,0.813034,-1.785093,2.931860,2.048742,-0.025249,0.0,0.0,1.0


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

log_reg = LogisticRegression()
pipeline = Pipeline([
    ('ct', ct),
    ('log_reg', log_reg)
])

pipeline.fit(X_train, y_train)

print('Training Score: ', pipeline.score(X_train, y_train))
print('Testing Score: ', pipeline.score(X_test, y_test))

Training Score:  0.800561797752809
Testing Score:  0.8100558659217877


In [10]:
pipeline.predict

<bound method Pipeline.predict of Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Sex', 'Embarked'])])),
                ('log_reg', LogisticRegression())])>

In [11]:
import pickle

pickle.dump(pipeline, open('pipeline.pkl', 'wb'))

In [12]:
new_data = X_test.iloc[0]
new_data

Pclass            3
Sex            male
Age            28.0
SibSp             1
Parch             1
Fare        15.2458
Embarked          C
Name: 709, dtype: object

In [13]:
pd.DataFrame(new_data).T

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,male,28.0,1,1,15.2458,C


In [14]:
model = pickle.load(open('pipeline.pkl', 'rb'))
pred = model.predict(pd.DataFrame(new_data).T)
if pred[0] == 0:
    print('Not survived')
else:
    print('Survived')


Not survived


In [15]:
%%writefile titanic_app.py

import streamlit as st
import pickle
import pandas as pd

st.title('Titanic Survival Prediction')

# Read Data
df = pd.read_csv('titanic_train.csv')
df = df.dropna()

# Take inputs
pclass = st.selectbox('Pclass', df.Pclass.unique())
sex = st.selectbox('Sex', df.Sex.unique())
age = st.number_input('Age')
sibsp = st.number_input('SibSp', df.SibSp.min(), df.SibSp.max())
parch = st.number_input('Parch', df.Parch.min(), df.Parch.max())
fare = st.number_input('Fare', df.Fare.min(), df.Fare.max())
embarked = st.selectbox('Embarked', df.Embarked.unique())

# Data Format
data = {
    'Pclass': pclass,
    'Sex': sex,
    'Age': age,
    'SibSp': sibsp,
    'Parch': parch,
    'Fare': fare,
    'Embarked': embarked
}

model = pickle.load(open('pipeline.pkl', 'rb'))
pred = model.predict(pd.DataFrame(data, index=[0]))

if st.button('Predict'):
    if pred[0] == 0:
        st.error('Not survived')
    else:
        st.success('Survived')

Overwriting titanic_app.py


In [16]:
!streamlit run titanic_app.py

^C


In [1]:
!pipreqs .

INFO: Successfully saved requirements file in .\requirements.txt
